# Description

This notebook will use the outcome of AFNI QC reports to decide which scans we keep and which scans we will not include in sucessive analyses.

The following ```get_ss_review_table``` command will generate two tables:

* ```review_all_scans.txt```: contains summary QC values for all scans that completed afni_proc.py
* ```review_keepers.txt```: contains information only for scans that pass out QC criteria.

Our QC criteria is:

1. Only include scans from the GE scanner, becuase these seem to have slice timing information properly encoded (as opposed to the Siemens ones that do not): ```-report_outliers 'censor fraction' GE 0.1```
2. Remove scans for which the EPI to Anat failed. Upon visual inspection we deem anat/EPI Dice <=.88 to reflect errors: ```-report_outliers 'anat/EPI mask Dice coef' LT 0.88```
3. Remove any scan that required an L/R Flip (None did, but nevertheless we leave the condition here): ```-report_outliers 'flip guess' EQ FLIP```
4. Remove any scan with dimensions different from those of the first scan (a few scans had voxel size slightly bigger (.001mm)). Most likely not an issue, but just to be on the safe side: ```-report_outliers 'orig voxel resolution' VARY```
5. Remove any scan with a fraction of censored TRs greater than 0.1 mm: ```-report_outliers 'censor fraction' GT 0.1```

```bash
ml afni
cd /data/SFIMJGC_HCP7T/BCBL2024/prcs_data
gen_ss_review_table.py -overwrite \
                       -report_outliers 'anat/EPI mask Dice coef' LT 0.88 \
                       -report_outliers 'slice timing pattern' EQ simult \
                       -report_outliers 'flip guess' EQ FLIP \
                       -report_outliers 'orig voxel counts' VARY \
                       -report_outliers 'orig voxel resolution' VARY \
                       -report_outliers 'TR' VARY \
                       -report_outliers 'num TRs per run' VARY \
                       -report_outliers 'average motion (per TR)' SHOW \
                       -report_outliers 'max motion displacement' SHOW \
                       -report_outliers 'max censored displacement' SHOW \
                       -report_outliers 'max motion displacement'   SHOW \
                       -report_outliers 'num TRs per run (applied)' SHOW \
                       -report_outliers 'TSNR average' SHOW \
                       -report_outliers 'global correlation (GCOR)' SHOW \
                       -report_outliers 'anat/templ mask Dice coef' SHOW \
                       -report_outliers 'censor fraction' SHOW \
                       -write_table review_all_scans.txt \
                       -write_outliers review_keepers.txt -show_keepers \
                       -infiles ./sub-*/D02_Preproc_fMRI_ses-?/out.ss_review.*
```

Not using this for now, as we want to have all possible motion here

                       -report_outliers 'censor fraction' GT 0.1 \


Data from this dataset was acquired in two different scanners:

* GE scanner --> Seems to have slice timing information available
* Siemens scanner --> Slice timing information is missing from the headers.

In [1]:
import pandas as pd
import numpy as np
import hvplot.pandas

In [2]:
report_summary_path  = '/data/SFIMJGC_HCP7T/BCBL2024/prcs_data/review_all_scans.txt'
report_keepers_path  = '/data/SFIMJGC_HCP7T/BCBL2024/prcs_data/review_keepers.txt'

# 1. Load the Full Report

Sometimes we will want to compare what we keep to the full sample. It is for these purposes that we load the full report

In [3]:
report_summary_df = pd.read_csv(report_summary_path, sep='\t')
report_summary_df = report_summary_df.infer_objects().drop(0)
# Correct Columns (e.g., removed UnNamed)
initial_col_name = list(report_summary_df.columns)
new_col_name     = []
for col in initial_col_name:
    if 'Unnamed' in col:
        new_col_name.append(new_col_name[-1])
    else:
        new_col_name.append(col)
report_summary_df.columns = new_col_name
# Assign Correct Type
report_summary_df['num TRs above mot limit']   = report_summary_df['num TRs above mot limit'].astype(int)
report_summary_df['TSNR average']              = report_summary_df['TSNR average'].astype(float)
report_summary_df['average motion (per TR)']   = report_summary_df['average motion (per TR)'].astype(float)
report_summary_df['max motion displacement']   = report_summary_df['max motion displacement'].astype(float)
report_summary_df['max censored displacement'] = report_summary_df['max censored displacement'].astype(float)
report_summary_df['anat/EPI mask Dice coef']   = report_summary_df['anat/EPI mask Dice coef'].astype(float)
report_summary_df['anat/templ mask Dice coef'] = report_summary_df['anat/templ mask Dice coef'].astype(float)
report_summary_df['global correlation (GCOR)'] = report_summary_df['global correlation (GCOR)'].astype(float)

print(report_summary_df.shape)
report_summary_df.head(5)

(596, 63)


,infile,subject ID,AFNI version,AFNI package,TR,TRs removed (per run),multiband level,slice timing pattern,echo times,echo times,...,anat/templ mask Dice coef,++ WARNING,++ WARNING,++ WARNING,++ WARNING,++ WARNING,++ WARNING,++ WARNING,++ WARNING,++ WARNING
1,./sub-01/D02_Preproc_fMRI_ses-1/out.ss_review....,sub-01,AFNI_24.3.04,linux_rocky_8,3.0,3,1,alt+z,13.7,30,...,0.995746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,./sub-01/D02_Preproc_fMRI_ses-2/out.ss_review....,sub-01,AFNI_24.3.04,linux_rocky_8,3.0,3,1,alt+z,13.7,30,...,0.995746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,./sub-02/D02_Preproc_fMRI_ses-1/out.ss_review....,sub-02,AFNI_24.3.04,linux_rocky_8,3.0,3,1,alt+z,13.7,30,...,0.994861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,./sub-02/D02_Preproc_fMRI_ses-2/out.ss_review....,sub-02,AFNI_24.3.04,linux_rocky_8,3.0,3,1,alt+z,13.7,30,...,0.994861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,./sub-03/D02_Preproc_fMRI_ses-1/out.ss_review....,sub-03,AFNI_24.3.04,linux_rocky_8,3.0,3,1,alt+z,13.7,30,...,0.995003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(list(report_summary_df.columns.unique()))

['infile', 'subject ID', 'AFNI version', 'AFNI package', 'TR', 'TRs removed (per run)', 'multiband level', 'slice timing pattern', 'echo times', 'num stim classes provided', 'final anatomy dset', 'final stats dset', 'final errts dset', 'orig voxel counts', 'orig voxel resolution', 'orig volume center', 'final voxel resolution', 'motion limit', 'num TRs above mot limit', 'average motion (per TR)', 'average censored motion', 'max motion displacement', 'max censored displacement', 'outlier limit', 'average outlier frac (TR)', 'num TRs above out limit', 'flip guess', 'num runs found', 'num TRs per run', 'num TRs per run (applied)', 'num TRs per run (censored)', 'fraction censored per run', 'TRs total (uncensored)', 'TRs total', 'degrees of freedom used', 'degrees of freedom left', 'final DF fraction', 'TRs censored', 'censor fraction', 'num regs of interest', 'TSNR average', 'global correlation (GCOR)', 'anat/EPI mask Dice coef', 'anat/templ mask Dice coef', '++ WARNING']


***

# 2. Load the list of scans that passed QC criteria

In [5]:
report_keepers_df = pd.read_csv(report_keepers_path, sep='\t')
outlier_criteria  = report_keepers_df.iloc[0]
report_keepers_df = report_keepers_df.drop(0)
report_keepers_df['average motion (per TR)']   = report_keepers_df['average motion (per TR)'].astype(float)
report_keepers_df['max motion displacement']   = report_keepers_df['max motion displacement'].astype(float)
report_keepers_df['max censored displacement'] = report_keepers_df['max censored displacement'].astype(float)
report_keepers_df['TSNR average']              = report_keepers_df['TSNR average'].astype(float)
report_keepers_df['TSNR average']              = report_keepers_df['TSNR average'].astype(float)
report_keepers_df['anat/EPI mask Dice coef']   = report_keepers_df['anat/EPI mask Dice coef'].astype(float)
report_keepers_df['anat/templ mask Dice coef'] = report_keepers_df['anat/templ mask Dice coef'].astype(float)
report_keepers_df['global correlation (GCOR)'] = report_keepers_df['global correlation (GCOR)'].astype(float)

print(report_keepers_df.shape)
report_keepers_df.head(5)

(434, 21)


,infile,anat/EPI mask Dice coef,slice timing pattern,flip guess,orig voxel counts,Unnamed: 5,Unnamed: 6,orig voxel resolution,Unnamed: 8,Unnamed: 9,...,num TRs per run,average motion (per TR),max motion displacement,max censored displacement,max motion displacement.1,num TRs per run (applied),TSNR average,global correlation (GCOR),anat/templ mask Dice coef,censor fraction
1,./sub-01/D02_Preproc_fMRI_ses-1/out.ss_review....,0.931619,alt+z,NO_FLIP,72,72,46,3.000000,3.000000,3.000000,...,201,0.026397,0.682144,0.682144,0.682144,201,224.328,0.007256,0.995746,0.000000
2,./sub-01/D02_Preproc_fMRI_ses-2/out.ss_review....,0.930440,alt+z,NO_FLIP,72,72,46,3.000000,3.000000,3.000000,...,201,0.025565,0.785410,0.785410,0.78541,201,216.095,0.005626,0.995746,0.000000
3,./sub-02/D02_Preproc_fMRI_ses-1/out.ss_review....,0.893903,alt+z,NO_FLIP,72,72,46,3.000000,3.000000,3.000000,...,201,0.031492,0.902465,0.902465,0.902465,199,237.958,0.009406,0.994861,0.009950
4,./sub-02/D02_Preproc_fMRI_ses-2/out.ss_review....,0.895153,alt+z,NO_FLIP,72,72,46,3.000000,3.000000,3.000000,...,201,0.063712,0.913374,0.913374,0.913374,201,237.967,0.009331,0.994861,0.000000
5,./sub-03/D02_Preproc_fMRI_ses-1/out.ss_review....,0.900406,alt+z,NO_FLIP,72,72,46,3.000000,3.000000,3.000000,...,201,0.154183,1.213150,1.111600,1.21315,114,254.041,0.018245,0.995003,0.432836


In [6]:
good_scan_ids = list(report_keepers_df['infile'])
print("++ Number of selected scans: %d scans" % len(good_scan_ids))

++ Number of selected scans: 434 scans


# 3. Check motion

In [7]:
report_keepers_df.hvplot.hist('average motion (per TR)', bins=np.linspace(0,0.4,50), label='keepers', normed=True, alpha=.5, title='Average Motion (per TR)') * \
report_summary_df['average motion (per TR)'].hvplot.hist('average motion (per TR)', bins=np.linspace(0,0.4,50), label='all', alpha=.5, normed=True)

:Overlay
   .Histogram.I  :Histogram   [average motion (per TR)]   (Frequency)
   .Histogram.II :Histogram   [average motion (per TR)]   (Frequency)

In [8]:
report_keepers_df.hvplot.hist('max motion displacement', bins=np.linspace(0,6,50), label='keepers', normed=True, alpha=.5, title='Maximum Motion Displacement') * \
report_summary_df['max motion displacement'].hvplot.hist('max motion displacement', bins=np.linspace(0,6,50), label='all', alpha=.5, normed=True)

:Overlay
   .Histogram.I  :Histogram   [max motion displacement]   (Frequency)
   .Histogram.II :Histogram   [max motion displacement]   (Frequency)

Most likely we will use 'max censored displacement' as the estimate of motion when doing the Powers et al. business. As according to AFNI Discourse (https://discuss.afni.nimh.nih.gov/t/max-motion-displacement-question/2754/3)... "The censored displacement just considers time points that were not removed, those that are still in the time series after censoring. It is more useful, since those are the time points that will be considered in the regression."

In [9]:
report_keepers_df.hvplot.scatter(x='max motion displacement', y='max censored displacement', hover_cols=['infile'])

:Scatter   [max motion displacement]   (max censored displacement,infile)

# 4. Check TSNR

In [10]:
report_keepers_df.hvplot.hist('TSNR average', title='Distribution of TSNR in final sample')

:Histogram   [TSNR average]   (Count)

# 5. Check Alignment Quality

In [11]:
report_keepers_df.hvplot.hist('anat/EPI mask Dice coef', title='Quality of EPI - Anat Overlap', normed=True, alpha=0.5, bins=np.linspace(.7,1,50)) * \
report_summary_df['anat/EPI mask Dice coef'].hvplot.hist('anat/EPI mask Dice coef', title='Quality of EPI - Anat Overlap', normed=True, alpha=0.5, bins=np.linspace(.7,1,50))

:Overlay
   .Histogram.I  :Histogram   [anat/EPI mask Dice coef]   (Frequency)
   .Histogram.II :Histogram   [anat/EPI mask Dice coef]   (Frequency)

In [12]:
report_keepers_df.hvplot.hist('anat/templ mask Dice coef', title='Quality of Transformation to MNI')

:Histogram   [anat/templ mask Dice coef]   (Count)

# 6. Check other metrics

In [13]:
report_keepers_df.hvplot.hist('global correlation (GCOR)', title='global correlation (GCOR)')

:Histogram   [global correlation (GCOR)]   (Count)

# 7. Write final list of scans to disk

In [14]:
sbj_idx = [i.split('/')[1] for i in report_keepers_df['infile'].values]
ses_idx = [i.split('/')[2].split('_')[-1] for i in report_keepers_df['infile'].values]
report_keepers_df.index = pd.MultiIndex.from_arrays([sbj_idx,ses_idx],names=['Subject','Session'])

In [15]:
report_keepers_df.to_csv('../../../resources/good_scans.txt')

In [16]:
gen_ss_review_scripts.py -show_uvar_dict

NameError: name 'gen_ss_review_scripts' is not defined